In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/My Drive/Colab Notebooks/NLP_Monsoon2020/Midsem_task1/

/content/drive/My Drive/Colab Notebooks/NLP_Monsoon2020/Midsem_task1


In [ ]:
ls

 assets/
 bert
 bert-model
 cache_dir/
'Constraint@AAAI2021 - COVID19 Fake News Detection in English results.csv'
'Constraint_English_Test - Sheet1.csv'
'Constraint_English_Train _Comma.csv'
'Constraint_English_Train _Comma.gsheet'
 Constraint_English_Train.xlsx
 Constraint_English_Val_Comma.csv
 Constraint_English_Val.xlsx
 model1.pb
 nlp_midsem_attention.ipynb
 nlp_midsem_task1_main.ipynb
 outputs/
 runs/
 variables/
 wandb/


In [ ]:
import json
import re
import csv
import nltk
import math
# import gensim = 3.8.3
import string
import numpy as np
import pandas as pd
from nltk.util import ngrams
# from num2words import num2words
from gensim.models import Word2Vec
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import sent_tokenize, word_tokenize

In [ ]:
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support

In [ ]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
train_data_df = pd.read_csv('Constraint_English_Train _Comma.csv', index_col=0, header=0)
val_data_df = pd.read_csv('Constraint_English_Val_Comma.csv', index_col=0, header=0)

train_data_df.replace(to_replace=['fake', 'real'], value=[1, 0], inplace=True)
print(train_data_df['label'].value_counts())

val_data_df.replace(to_replace=['fake','real'], value=[1,0],inplace=True)
print(val_data_df['label'].value_counts())

test_data_df = pd.read_csv('Constraint_English_Test - Sheet1.csv')

# x_train = train_data['tweet']
# y_train = train_data['label']

# x_val = val_data['tweet']
# y_val = val_data['label']

0    3360
1    3060
Name: label, dtype: int64
0    1120
1    1020
Name: label, dtype: int64


In [ ]:
print(train_data_df.head())
print(val_data_df.head())
print(test_data_df.head())

                                                tweet  label
id                                                          
1   The CDC currently reports 99031 deaths. In gen...      0
2   States reported 1121 deaths a small rise from ...      0
3   Politically Correct Woman (Almost) Uses Pandem...      1
4   #IndiaFightsCorona: We have 1524 #COVID testin...      0
5   Populous states can generate large case counts...      0
                                                tweet  label
id                                                          
1   Chinese converting to Islam after realising th...      1
2   11 out of 13 people (from the Diamond Princess...      1
3   COVID-19 Is Caused By A Bacterium, Not Virus A...      1
4   Mike Pence in RNC speech praises Donald Trump’...      1
5   6/10 Sky's @EdConwaySky explains the latest #C...      0
   id                                              tweet
0   1  Our daily update is published. States reported...
1   2             Alfalfa is the

In [ ]:
train_data = train_data_df.to_numpy()
val_data = val_data_df.to_numpy()


In [ ]:

test_data = test_data_df['tweet'].to_numpy()
for i in range(len(test_data)):
  test_data[i] = [test_data[i]]

test_data = np.array(test_data)

In [ ]:
print(train_data.shape)
print(train_data[1595])
print(test_data.shape)
print(test_data[0])

(6420, 2)
['https://t.co/HcGWx8897G California fires, camel racing, coronavirus tribute; The World in Photos, Sept. 28 https://t.co/vmEP0HhNbC https://t.co/4KvZ8FEWPX'
 1]
(2140,)
['Our daily update is published. States reported 734k tests 39k new cases and 532 deaths. Current hospitalizations fell below 30k for the first time since June 22. https://t.co/wzSYMe0Sht']


In [ ]:
!pip install emoji

In [ ]:
import emoji
def clean_text(text):
  # print(text)
  text = re.sub(r'#','', text)
  # Remove HTML special entities (e.g. &amp;)
  # print(text)
  text = re.sub(r'\&\w*;', '', text)
  # Remove tickers
  # text = re.sub(r'\$\w*', '', text)
  # Remove hyperlinks
  # print(text)
  # print('hello')
  # text = re.sub(r'https?:\/\/.*\/*', '', text)
  # Remove whitespace (including new line characters)
  # print(text)
  # print(text)
  # print('hello')
  text = re.sub(r'\s\s+','', text)
  text = re.sub(r'[ ]{2, }',' ',text)
  # Remove URL, RT, mention(@)
  # print(text)
  # print(text)
  # print('hello')
  text=  re.sub(r'http(\S)+', '',text)
  text=  re.sub(r'http ...', '',text)
  text=  re.sub(r'(RT|rt)[ ]*@[ ]*[\S]+','',text)
  text=  re.sub(r'RT[ ]?@','',text)
  text = re.sub(r'@[\S]+','',text)
  text = re.sub(r'[^\w\s]', '', text) 
  # print(text)
  text = re.sub('\\n', ' ',text)
  text = re.sub('\\r', ' ',text)
  text = re.sub('\\t', ' ',text)
  text = re.sub('<[^>]*>', ' ',text)

  # Remove emoji's
  text = emoji.demojize(text)
  text = text.replace(":"," ")
  text = ' '.join(text.split()) 
  text = re.sub("([^\x00-\x7F])+"," ",text)
  
  return text

def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(w) for w in text]

In [ ]:
print(train_data[1595])
print(clean_text(train_data[1595][0]))

['https://t.co/HcGWx8897G California fires, camel racing, coronavirus tribute; The World in Photos, Sept. 28 https://t.co/vmEP0HhNbC https://t.co/4KvZ8FEWPX'
 1]
California fires camel racing coronavirus tribute The World in Photos Sept 28


In [ ]:
def parser(data):
  for i in range(len(data)):
    data[i][0] = clean_text(data[i][0])
    if(data[i][0]==''):
      print(i)
    tokens  = word_tokenize(data[i][0])
    lemmatizer = WordNetLemmatizer()
    data[i][0] = [lemmatizer.lemmatize(w) for w in tokens]
    # print(data[i][0])
    

In [ ]:
parser(train_data)
parser(val_data)


In [ ]:
parser(test_data)

In [ ]:
print(train_data[0][0])
print(val_data[0][0])
print(test_data[0][0])

['The', 'CDC', 'currently', 'report', '99031', 'death', 'In', 'general', 'the', 'discrepancy', 'in', 'death', 'count', 'between', 'different', 'source', 'are', 'small', 'and', 'explicable', 'The', 'death', 'toll', 'stand', 'at', 'roughly', '100000', 'people', 'today']
['Chinese', 'converting', 'to', 'Islam', 'after', 'realising', 'that', 'no', 'muslim', 'wa', 'affected', 'by', 'Coronavirus', 'COVD19', 'in', 'the', 'country']
['Our', 'daily', 'update', 'is', 'published', 'States', 'reported', '734k', 'test', '39k', 'new', 'case', 'and', '532', 'death', 'Current', 'hospitalization', 'fell', 'below', '30k', 'for', 'the', 'first', 'time', 'since', 'June', '22']


In [ ]:
vocab = set()

def create_vocab(vocab_set, data):
  for sample in data:
    for token in sample[0]:
      if(token not in vocab_set):
        vocab_set.add(token)
  print(len(vocab_set))

create_vocab(vocab,train_data)
create_vocab(vocab,val_data)
create_vocab(vocab,test_data)

17713
20633
23208


In [ ]:
glove_dict = {}

def create_glove_dict(gdict, data):
  f = open("/content/drive/My Drive/NLP_project/atten/glove.840B.300d.txt",'r',encoding='utf-8')
  i = 0
  for line in f:
    values = line.split()
    word = values[0]
    if(word in vocab):
      st = 1
      if(len(values)>301):
        st+=len(values) - 301
      vector = np.asarray(values[st:],'float32')
      gdict[word] = vector

create_glove_dict(glove_dict,train_data)
create_glove_dict(glove_dict,val_data)
create_glove_dict(glove_dict,test_data)

# with open("glove.840B.300d.txt", 'r', encoding="utf-8") as f:
#   i=0
#   for line in f:
#     i+=1
#     values = line.split()
#     word = values[0]
#     if(word in vocab):
#       st = 1
#       if(len(values)>301):
#         st+= len(values) - 301
#       vector = np.asarray(values[st:],'float32')
#       glove_dict[word] = vector

print(len(glove_dict))

18031


In [ ]:
import pickle

In [ ]:
with open('glove_dict_file','wb') as f:
  pickle.dump(glove_dict,f)

In [ ]:
with open('glove_dict_file','rb') as f:
  glove_dict = pickle.load(f)

In [ ]:
print(len(glove_dict))

18031


In [ ]:
# mu, sigma = 0, 0.1
# s = np.random.normal(mu, sigma, 300)
# s = np.asarray(s,'float32')
s = [0.0]*300
# print(s.shape)
print(s)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [ ]:
print(type(s[0]))

<class 'float'>


In [ ]:
for word in vocab:
  if(word not in glove_dict):
    glove_dict[word] = np.asarray(s,'float32')

print(len(vocab))
print(len(glove_dict))

23208
23208


In [ ]:
!pip install texttable


In [ ]:
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install torch-scatter==latest+{CUDA}     -f https://pytorch-geometric.com/whl/torch-{TORCH}.html
!pip install torch-sparse==latest+{CUDA}      -f https://pytorch-geometric.com/whl/torch-{TORCH}.html
!pip install torch-cluster==latest+{CUDA}     -f https://pytorch-geometric.com/whl/torch-{TORCH}.html
!pip install torch-spline-conv==latest+{CUDA} -f https://pytorch-geometric.com/whl/torch-{TORCH}.html
!pip install torch-geometric

Looking in links: https://pytorch-geometric.com/whl/torch-1.7.0.html
     |████████████████████████████████| 11.9MB 2.9MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.7.0.html
     |████████████████████████████████| 24.3MB 2.7MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.7.0.html
     |████████████████████████████████| 21.5MB 663kB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.7.0.html
     |████████████████████████████████| 6.4MB 598kB/s 
     |████████████████████████████████| 194kB 4.3MB/s 
     |████████████████████████████████| 235kB 7.9MB/s 
     |████████████████████████████████| 2.2MB 9.5MB/s 
     |████████████████████████████████| 51kB 5.5MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-1.6.3-cp36-none-any.whl size=322720 sha256=99be4c812dc89e9f2ab29b3ec547d70e8c2bb72d08da7f9bfd33fd0aa8b0aa27
  Stored in directory: /root/.cache/pip/wheels/6d/47/1e/0af8ce3e21783c3e584c22502011a3367c091694eebc50a971
Succe

In [ ]:
# import torch

class AttentionModule(torch.nn.Module):
    
    def __init__(self, args):
        
        super(AttentionModule, self).__init__()
        self.args = args
        self.filters_3 = 300
        self.filters_4 = 64
        self.setup_weights()
        self.init_parameters()

    def setup_weights(self):
        
        self.weight_matrix = torch.nn.Parameter(torch.Tensor(self.filters_3,
                                                             self.filters_3))

    def init_parameters(self):
       
        torch.nn.init.xavier_uniform_(self.weight_matrix)

    def forward(self, embedding):
        # print(type(embedding))
        global_context = torch.mean(torch.matmul(embedding, self.weight_matrix), dim=0)
        transformed_global = torch.tanh(global_context)
        # print(type(transformed_global))
        # print(transformed_global.shape)
        sigmoid_scores = torch.sigmoid(torch.mm(embedding, transformed_global.view(-1, 1)))
        representation = torch.mm(torch.t(embedding), sigmoid_scores)
        return representation



In [ ]:
class Model(torch.nn.Module):
    def __init__(self, args):
        
        super(Model, self).__init__()
        self.args = args
        self.tensor_neurons = 16
        self.bins = 16
        self.histogram = False
        self.setup_layers()

    def setup_layers(self):
        
        
        self.attention = AttentionModule(self.args)
      
        
        self.fully_connected_first = torch.nn.Linear(300,200)

       

        self.fully_connected_second = torch.nn.Linear(200,100)
        self.fully_connected_third = torch.nn.Linear(100,50)
        
        self.fully_connected_fourth = torch.nn.Linear(50,25)
        # self.fully_connected_fifth = torch.nn.Linear(16,8)
        self.scoring_layer = torch.nn.Linear(25,1)

  

    def forward(self, data):
        
        utter_feature = torch.from_numpy(np.array(data[0]))
        utter_atten = self.attention(utter_feature)
        scores = utter_atten
        scores = torch.t(scores)
        # print(scores.shape)

        score1 = torch.nn.functional.relu(self.fully_connected_first(scores))
        score2 = torch.nn.functional.relu(self.fully_connected_second(score1))
        score3 = torch.nn.functional.relu(self.fully_connected_third(score2))
        score4 = torch.nn.functional.relu(self.fully_connected_fourth(score3))
        # score5 = torch.nn.functional.relu(self.fully_connected_fifth(score4))
        score = torch.sigmoid(self.scoring_layer(score4))
        
        return score,score4


In [ ]:
class Trainer(object):
    
    def __init__(self, args,train_data,val_data,test_data,glove_dict):
        
        self.args = args
        self.train_data = train_data
        self.val_data = val_data
        self.test_data = test_data
        self.glove_dict = glove_dict
        self.embedding_gen()
        self.setup_model()
 
    def setup_model(self):
        
        self.model = Model(self.args)
 
    def embedding_gen(self):

      print("\nGenerating for train\n")
      label_train_data = []
      for sample in self.train_data[:,1]:
        label = sample
        label_train_data.append(label)
      label_train_data = np.array(label_train_data)

      tweet_train_data = []
      for sample in self.train_data[:,0]:
        feature_vec = []
        if(len(sample)<=1):
          print(sample)
        for token in sample:
          feature_vec.append(self.glove_dict[token])
        tweet_train_data.append(np.array(feature_vec))
      tweet_train_data = np.array(tweet_train_data)


      print("\nGenerating for val\n")

      label_val_data = []
      for sample in self.val_data[:,1]:
        label = sample
        label_val_data.append(label)
      label_val_data = np.array(label_val_data)

      tweet_val_data = []
      for sample in self.val_data[:,0]:
        feature_vec = []
        for token in sample:
          feature_vec.append(self.glove_dict[token])
        tweet_val_data.append(np.array(feature_vec))
      tweet_val_data = np.array(tweet_val_data)

  

      print("\nGenerating for test\n")

      tweet_test_data = []
      for sample in self.val_data[:,0]:
        feature_vec = []
        for token in sample:
          feature_vec.append(self.glove_dict[token])
        tweet_test_data.append(np.array(feature_vec))
      tweet_test_data = np.array(tweet_test_data)

      self.train_dataset = []
      self.val_dataset = []
      self.test_dataset = []
      
      for i in range(len(self.train_data)):
        self.train_dataset.append([tweet_train_data[i],label_train_data[i]])
      for i in range(len(self.val_data)):
        self.val_dataset.append([tweet_val_data[i],label_val_data[i]])
      for i in range(len(self.test_data)):
        self.test_dataset.append([tweet_test_data[i]])
      
      self.train_dataset = np.array(self.train_dataset)
      self.val_dataset = np.array(self.val_dataset)
      self.test_dataset = np.array(self.test_dataset)

      print(self.train_dataset.shape)
      print(self.val_dataset.shape)
      print(self.test_dataset.shape)
 
            
    def create_batches(self):
       
        
        mod_batches = []
        np.random.shuffle(self.train_dataset)
        
        ind = 0
        while(ind<len(self.train_dataset)):  
          batches = []
          for num in range(10):
            batches.append(self.train_dataset[ind])
            ind += 1
           
          mod_batches.append(batches)
        return mod_batches
 
    def process_batch(self, batch):
        
        self.optimizer.zero_grad()
        losses = 0
        # print('process_batch')
        # it = 0
        for sample in batch:
            data = sample[:len(sample)-1]
            label = sample[-1]
            result = torch.from_numpy(np.array(label).reshape(1,1)).view(-1).float() 
            prediction,_ = self.model(data)
            losses = losses + torch.nn.functional.mse_loss(result, prediction)
        losses.backward(retain_graph=True)
        self.optimizer.step()
        loss = losses.item()
        return loss
 
    def fit(self):
      
 
        print("\nModel training.\n")
        self.optimizer = torch.optim.Adam(self.model.parameters(),
                                          lr=0.001,
                                          weight_decay=5*10**-4)
 
        self.model.train()
        
        epochs = 150
        min_loss = 10000
        min_loss_epoch = 0
        self.min_loss_model = self.model
        for epoch in range(epochs):
 
            batches = self.create_batches()
            
            self.loss_sum = 0
            main_index = 0

            for batch in batches:
                loss_score = self.process_batch(batch)
                
                main_index = main_index + len(batch)
                # print(main_index)
                self.loss_sum = self.loss_sum + loss_score * len(batch)
                # self.loss_sum = self.loss_sum + loss_score
                loss = self.loss_sum/main_index
                
                
            # print(loss)
            if loss < min_loss:
                    min_loss = loss
                    min_loss_epoch = epoch
                    self.min_loss_model = self.model
                    print('min_loss: ',min_loss)
                    print(min_loss_epoch)
                    # print(self.min_loss_model)
            
 
        svm_feature = []      
        for sample in self.train_dataset:
          data = sample[:len(sample)-1]
          _,feature = self.min_loss_model(data)
          svm_feature.append(feature.tolist()[0])
 
        svm_target = []
        for sample in self.train_dataset:
          result = sample[-1]
          svm_target.append(result)


        cnt0 = 0
        cnt1 = 0
        for i in svm_target:
          if(i==0):
            cnt0+=1
          else:
            cnt1+=1
        print(cnt0)
        print(cnt1)
 
        
        X = np.array(svm_feature)
        y = np.array(svm_target)
        print(y.shape)
        print(np.unique(y))
        from sklearn import svm
        from sklearn.metrics import confusion_matrix,accuracy_score
        self.svm_rbf_model = svm.SVC(kernel = 'rbf',gamma = 'scale', decision_function_shape = 'ovr')
        self.svm_rbf_model.fit(X,y)
        print("svm_model_trained")
        #now adding SVM rbf classifier for removing step function
        
    
 
    
    def score(self):
        
        print("\n\nModel evaluation.\n")
        
 
        model1 = self.min_loss_model
        model1.eval()
        self.scores = []
 
        predictions = []
        true_label = []
        for sample in self.val_dataset:
            
            data = sample[:len(sample)-1]
            true_label.append(sample[-1])
            _,prediction = model1(data)
            predict = self.svm_rbf_model.predict(np.array(prediction.tolist())) 
            predictions.append(predict)
        
        return true_label,predictions,model1,self.svm_rbf_model
 
       

In [ ]:
# print(type(data))
args = 'ut-sp-ct-cp-sh'
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from tqdm import tqdm, trange
import pickle
from sklearn.metrics import precision_recall_fscore_support, classification_report
# skf = StratifiedKFold(n_splits=5, shuffle=True)
# kf = KFold(n_splits=5)
# kf.get_n_splits(data)
results = []
accur = []
# print(type(df))
max_f1 = 0
# for train_index, test_index in skf.split(data,df['sarcasm'].to_numpy()):
# for train_index, test_index in kf.split(data):  
  # train_data, test_data = data[train_index], data[test_index]
  # print(train_data.shape)
  # print(test_data.shape)

trainer = Trainer(args,train_data,val_data,test_data,glove_dict)
trainer.fit()
y_test,y_pred,atten_model,svm_model = (trainer.score())
  # filename_atten = args+'_atten_model_50.sav'
  # filename_svm = args+'_svm_model_50.sav'
  # filename_atten = args+'_atten_model_75.sav'
  # filename_svm = args+'_svm_model_75.sav'
  # filename_atten = args+'_atten_model_100.sav'
  # filename_svm = args+'_svm_model_100.sav'

  # filename_atten = args+'_atten_model_150.sav'
  # filename_svm = args+'_svm_model_150.sav'
accur.append(accuracy_score(y_test, y_pred))
results.append(precision_recall_fscore_support(y_test, y_pred, average='weighted'))

  # if(max_f1 < results[-1][2]):
  #   max_f1 = results[-1][2]
  #   pickle.dump(atten_model, open(filename_atten, 'wb'))
  #   pickle.dump(svm_model,open(filename_svm,'wb'))
  
print(accur[-1])
print(classification_report(y_test, y_pred))
  

# avg = [0,0,0,0]
# for i in range(5):
#     x, y, z, _ = results[i]
#     avg[0] += x
#     avg[1] += y
#     avg[2] += z
#     avg[3] += accur[i]
# avg[0]/=5
# avg[1]/=5
# avg[2]/=5
# avg[3]/=5


# print(f"Avg Accuracy: {avg[3]:.3f}")
# print(f"Avg weighted precision: {avg[0]:.3f} :: Avg weighted recall: {avg[1]:.3f} :: Avg weighted F1: {avg[2]:.3f}")






Generating for train

['StopTheSpreadOfCoronaLockdown21Corona']

Generating for val


Generating for test

(6420, 2)
(2140, 2)
(2140, 1)

Model training.



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:112: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.


min_loss:  0.963976062072758
0
min_loss:  0.5734441211013214
1
min_loss:  0.43563952836030717
2
min_loss:  0.3588319336181304
3
min_loss:  0.2926749471938529
4
min_loss:  0.2457936442664212
5
min_loss:  0.23815690890326555
6
min_loss:  0.23061507180987512
7
min_loss:  0.21113455520771093
9
min_loss:  0.17505056217002063
10
min_loss:  0.1604042697115695
12
min_loss:  0.13681281380948185
14
min_loss:  0.11711085412185743
17
min_loss:  0.11335211210957988
21
min_loss:  0.09302018147823529
27
min_loss:  0.0759657920951073
43
min_loss:  0.06685988406237085
53
min_loss:  0.061526685086909144
62
min_loss:  0.059466756419498855
92
min_loss:  0.05260729492778089
100
min_loss:  0.05062330963677242
111
min_loss:  0.04755152160084596
112
3360
3060
(6420,)
[0 1]
svm_model_trained


Model evaluation.

0.9065420560747663
              precision    recall  f1-score   support

           0       0.90      0.92      0.91      1120
           1       0.91      0.89      0.90      1020

    accuracy      